In [1]:
# ! conda install -c conda-forge python-annoy
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
tqdm.pandas()
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
import numpy as np
from operator import itemgetter

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
TOPN_candidate = 100
SETS = [2, 3]
VER = 6
SUBSETNUM = 2

note = 'covisit_20_20_20_newSuggester2_Covwgt_timeWgt'

predTypes = ['clicks', 'carts', 'orders']
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [4]:
def load_df(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [5]:
for SET in SETS:
    # candidatesSavePath = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{note}/'
    candidatesSavePath = outputPath + f'data4xgb_set{SET}_top_{TOPN_candidate}_{note}/'
    candidatesSavePath_cfSim = outputPath + f'data4xgb_set{SET}_top_{TOPN_candidate}_{note}_cfSim/'
    ! mkdir {candidatesSavePath_cfSim}
    cf_save_path = Path(f'../output/newSplited/cf_matrix/set_{SET}')
    itemSimMatrix = np.load(cf_save_path / 'itemSimMatrix.npy', allow_pickle='TRUE').item()
    aid2aidCfScore = {
        'aid_x': [],
        'aid_y': [],
        'sim': [],
        }
    for aid_x in tqdm(itemSimMatrix):
        for aid_y, score in sorted(itemSimMatrix[aid_x].items(), key=itemgetter(1), reverse=True)[1:21]:
            aid2aidCfScore['aid_x'].append(aid_x)
            aid2aidCfScore['aid_y'].append(aid_y)
            aid2aidCfScore['sim'].append(score)
    aid2aidCfScore = pd.DataFrame(aid2aidCfScore)

    if SET == 1:
        val_A = load_df('../input/split_2-1_pqt/test_parquets/*')
    elif SET == 2:
        val_A = pd.read_parquet('../input/splited/test.parquet')
    elif SET == 3:
        val_A = load_df('../input/parquets/test_parquets/*')
    val_A = val_A.drop_duplicates((['session', 'aid']))

    for predType in predTypes:
        print(predType)
        for s in range(SUBSETNUM):     
            print('sub:', s)    
            candidate = pd.read_parquet(candidatesSavePath + f'data4xgb_{predType}_{s}.pqt')
            candidate['session'] = candidate.session.astype('int32')
            df = val_A.merge(candidate[['session', 'aid']],on='session')

            df = df.merge(aid2aidCfScore, on=['aid_x', 'aid_y'], how='left').fillna(0)
            df = df.groupby(['session', 'aid_y']).sim.mean().reset_index()
            df.columns = ['session', 'aid', 'cfSim']

            candidate = candidate.merge(df, on=['session', 'aid'], how='left')
            candidate.to_parquet(candidatesSavePath_cfSim + f'data4xgb_{predType}_{s}.pqt')

A Jupyter Widget

clicks
sub: 0
sub: 1
carts
sub: 0
sub: 1
orders
sub: 0
sub: 1


A Jupyter Widget

clicks
sub: 0
sub: 1
carts
sub: 0
sub: 1
orders
sub: 0
sub: 1


In [6]:
candidate

,session,aid,covWgt_t,covWgt_b2b_t,covWgt_cl2or_t,order_by_rule,item_item_count,item_user_count,item_buy_ratio,item_type_std,...,user_type_std,user_ts_mean,user_ts_std,user_clicked_cnt,user_carted_cnt,user_ordered_cnt,cnt_clicked,cnt_carted,cnt_ordered,cfSim
0,13800404,957043,9.581833,0.0,0.0,49,0.038333,0.038396,0.104955,0.351911,...,0.48795,310656.0,414.692535,0.011547,0.009132,0.0,0.0,0.0,0.0,0.016197
1,13800404,479834,9.235502,0.0,0.0,48,0.119242,0.092384,0.144438,0.417210,...,0.48795,310656.0,414.692535,0.011547,0.009132,0.0,0.0,0.0,0.0,0.031237
2,13800404,521661,6.926626,0.0,0.0,47,0.004439,0.004781,0.091057,0.330089,...,0.48795,310656.0,414.692535,0.011547,0.009132,0.0,0.0,0.0,0.0,0.018471
3,13800404,1485090,7.723994,0.0,0.0,46,0.001690,0.002102,0.122881,0.419911,...,0.48795,310656.0,414.692535,0.011547,0.009132,0.0,0.0,0.0,0.0,0.080448
4,13800404,973371,7.612052,0.0,0.0,45,0.001277,0.001525,0.100559,0.353076,...,0.48795,310656.0,414.692535,0.011547,0.009132,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77117745,14571581,272744,0.000000,0.0,0.0,4,0.040864,0.047435,0.100763,0.353620,...,0.00000,604800.0,0.000000,0.002309,0.000000,0.0,0.0,0.0,0.0,0.000000
77117746,14571581,165160,0.000000,0.0,0.0,3,0.056785,0.063000,0.344867,0.613090,...,0.00000,604800.0,0.000000,0.002309,0.000000,0.0,0.0,0.0,0.0,0.000000
77117747,14571581,1708326,0.000000,0.0,0.0,2,0.137904,0.103882,0.173906,0.437037,...,0.00000,604800.0,0.000000,0.002309,0.000000,0.0,0.0,0.0,0.0,0.000000
77117748,14571581,1722991,0.000000,0.0,0.0,1,0.254078,0.224922,0.120058,0.387012,...,0.00000,604800.0,0.000000,0.002309,0.000000,0.0,0.0,0.0,0.0,0.000000
